In [1]:
from tqdm import tqdm
import requests  
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

## IMDb

In [2]:
def get_page_IMDB(p):
    
    url = 'https://www.imdb.com/search/title/?title_type=feature&sort=num_votes,desc&start={}&ref_=adv_nxt'.format(50*p+1)
    
    response = requests.get(url)
    
    tree = BeautifulSoup(response.content, 'html.parser')
    
    movies = tree.find_all('div', {'class': 'lister-item'})
    
    infaIMDB = [ ]
    d = {}

    for movie in movies:
        try:
            d = {'title': movie.h3.a.get_text(),
                 'year': int(''.join(filter(str.isdigit, movie.select('.lister-item-year')[0].get_text().strip()))),
                 'runtime': int(''.join(filter(str.isdigit, movie.select('.runtime')[0].get_text().strip()))),
                 'genre': movie.select('.genre')[0].get_text().strip().split(', '),
                 'imdb_rating': float(movie.strong.get_text()), 
                 'votes': int(''.join(filter(str.isdigit, 
                                             movie.select('.sort-num_votes-visible')[0].get_text().strip().split()[1])))}
            try:
                d['certificate'] = movie.select('.certificate')[0].get_text().strip()
            except:
                d['certificate'] = 'NaN'
            try: 
                d['metascore'] = int(movie.select('.ratings-metascore')[0].get_text().strip().split()[0])
            except:
                d['metascore'] = 0
            try: 
                d['gross'] = int(''.join(filter(str.isdigit, 
                                            movie.select('.sort-num_votes-visible')[0].select('span')[4].get('data-value'))))
            except:
                d['gross'] = 0
                
            infaIMDB.append(d)

        except:
            continue

    return infaIMDB

In [3]:
infaIMDB = []

for p in tqdm(range(0,1)):
    infaIMDB.extend(get_page_IMDB(p))

100%|██████| 1/1 [00:01<00:00,  1.38s/it]


In [4]:
dfIMDB = pd.DataFrame(infaIMDB)
print(dfIMDB.shape)
dfIMDB.head(10)

(50, 9)


,title,year,runtime,genre,imdb_rating,votes,certificate,metascore,gross
0,Побег из Шоушенка,1994,142,[Drama],9.3,2751184,16+,82,28341469
1,Тёмный рыцарь,2008,152,"[Action, Crime, Drama]",9.0,2723913,14+,84,534858444
2,Начало,2010,148,"[Action, Adventure, Sci-Fi]",8.8,2417730,12+,74,292576195
3,Бойцовский клуб,1999,139,[Drama],8.8,2190763,18+,66,37030102
4,Форрест Гамп,1994,142,"[Drama, Romance]",8.8,2140265,0+,82,330252182
5,Криминальное чтиво,1994,154,"[Crime, Drama]",8.9,2113490,18+,95,107928762
6,Матрица,1999,136,"[Action, Sci-Fi]",8.7,1961329,16+,73,171479930
7,Властелин колец: Братство кольца,2001,178,"[Action, Adventure, Drama]",8.8,1918844,12+,92,315544750
8,Интерстеллар,2014,169,"[Adventure, Drama, Sci-Fi]",8.7,1916682,12+,74,188020017
9,Крёстный отец,1972,175,"[Crime, Drama]",9.2,1913731,16+,100,134966411


## Кинопоиск

In [5]:
def get_page_KP(p):
    
    url = 'https://www.kinopoisk.ru/top/navigator/m_act[rating]/1%3A/m_act[is_film]/on/order/num_vote/page/{}/#results'.format(p)
    
    from fake_useragent import UserAgent
    UserAgent().chrome
    r = requests.get(url, headers={'User-Agent': UserAgent().chrome})
    
    sleep(50)
    tree = BeautifulSoup(r.text, "lxml")
    
    movies = tree.find_all('div', {'class': 'item'})
    
    infaKP = [ ]
    d = {}

    for movie in movies:
        try:
            d = {'title': movie.find_all('div', {'class': 'name'})[0].select('a')[0].get_text().split('(')[0].strip(),
                 'year': int(movie.find_all('div', {'class': 'name'})[0].select('span')[0].get_text().strip().split(')')[0].split('(')[1]),
                 'country': movie.find_all('span', 
                            {'class': 'gray_text'})[0].get_text().strip().replace('...', '').replace(',', '').split()[0],
                'director': movie.find_all('a', {'class': "lined"})[0].get_text(),
                'actor_1': movie.find_all('a', {'class': "lined"})[1].get_text(),
                 'actor_2': movie.find_all('a', {'class': "lined"})[2].get_text(),
                 'ratingKP': float(movie.find_all('div', {'class': "numVote"})[0].get_text().split()[0])}
           
            infaKP.append(d)
            
        except:
            pass

    return infaKP

In [6]:
infaKP = []
d = {}

for p in tqdm(range(1,2)):
    infaKP.extend(get_page_KP(p))

100%|██████| 1/1 [00:52<00:00, 52.11s/it]


In [7]:
dfKP = pd.DataFrame(infaKP)
print(dfKP.shape)
dfKP.head(10)

(50, 7)


,title,year,country,director,actor_1,actor_2,ratingKP
0,1+1,2011,Франция,Оливье Накаш,Франсуа Клюзе,Омар Си,8.806
1,Джентльмены,2019,Великобритания,Гай Ричи,Мэттью МакКонахи,Чарли Ханнэм,8.549
2,Волк с Уолл-стрит,2013,США,Мартин Скорсезе,Леонардо ДиКаприо,Джона Хилл,7.974
3,Гнев человеческий,2021,Великобритания,Гай Ричи,Джейсон Стэйтем,Холт Маккэллани,7.591
4,Брат,1997,Россия,Алексей Балабанов,Сергей Бодров мл.,Виктор Сухоруков,8.295
5,Один дома,1990,США,Крис Коламбус,Маколей Калкин,Джо Пеши,8.275
6,Аватар,2009,США,Джеймс Кэмерон,Сэм Уортингтон,Зои Салдана,7.971
7,Побег из Шоушенка,1994,США,Фрэнк Дарабонт,Тим Роббинс,Морган Фриман,9.108
8,Начало,2010,США,Кристофер Нолан,Леонардо ДиКаприо,Джозеф Гордон-Левитт,8.663
9,Зеленая миля,1999,США,Фрэнк Дарабонт,Том Хэнкс,Дэвид Морс,9.073


## 1000 звезд

In [8]:
def get_page_stars(p):
    
    url = 'https://www.imdb.com/list/ls058011111/?sort=list_order,asc&mode=detail&page={}'.format(p)
    
    response = requests.get(url)
    
    tree = BeautifulSoup(response.content, 'html.parser')
    
    stars = tree.find_all('div', {'class': 'lister-item-content'})
    
    stars_infa = [ ]

    for star in stars:
        try:
            stars_infa.append({'index': star.find_all('span', {'class': 'lister-item-index'})[0].get_text().split('.')[0],
                           'name': star.a.get_text().strip()}) 

        except:
            continue

    return stars_infa

In [9]:
stars_infa = []
d = {}

for p in tqdm(range(1,2)):
    stars_infa.extend(get_page_stars(p))

100%|██████| 1/1 [00:03<00:00,  3.75s/it]


In [10]:
dfstars = pd.DataFrame(stars_infa)
print(dfstars.shape)
dfstars.head(10)

(100, 2)


,index,name
0,1,Robert De Niro
1,2,Jack Nicholson
2,3,Marlon Brando
3,4,Denzel Washington
4,5,Katharine Hepburn
5,6,Humphrey Bogart
6,7,Meryl Streep
7,8,Daniel Day-Lewis
8,9,Sidney Poitier
9,10,Clark Gable
